In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import sys
import itertools
import random
import torch
import os
import signal
import gc
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Conv2DTranspose, Activation, MaxPool2D, UpSampling2D,Reshape, Flatten, Dense, Dropout,GlobalAveragePooling2D
from keras import backend as K
from keras.utils import plot_model
from numba import cuda,jit
import multiprocessing as mp
from multiprocessing import Pool, cpu_count
import psutil

np.set_printoptions(threshold=sys.maxsize,linewidth=512)
#np.set_printoptions(threshold=256)

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))
 
#여백 줄이기
display(HTML("<style>.prompt { min-width: 10ex !important; }</style>"))

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

In [3]:
##################################################################################################################

In [5]:
#np.save("./dataset2/1000/input100k.npy",r_input)
#np.save("./dataset2/1000/output100k.npy",r_output)

r_input=np.load("./dataset3/input500k.npy")
r_output=np.load("./dataset3/output500k_num_class.npy")



print(r_input.shape)
print(r_output.shape)
#print(test_input.shape)
#print(test_output.shape)



(500000, 32, 64, 3)
(500000, 10)


In [10]:
# Define the input shape
input_shape = (32, 64, 3)

# Define the model
model3 = tf.keras.models.Sequential([
    
  # Flatten the input to a 1D array
    tf.keras.layers.Flatten(input_shape=input_shape),
  # Add a fully connected (dense) layer with 128 units and ReLU activation
    tf.keras.layers.Dense(6144, activation='relu'),
  # Add another fully connected (dense) layer with 64 units and ReLU activation
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
  # Add a final fully connected (dense) layer with 10 units and softmax activation
  tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model with categorical cross-entropy loss and Adam optimizer
model3.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model3.summary()






Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 6144)              0         
                                                                 
 dense_9 (Dense)             (None, 6144)              37754880  
                                                                 
 dense_10 (Dense)            (None, 2048)              12584960  
                                                                 
 dense_11 (Dense)            (None, 1024)              2098176   
                                                                 
 dense_12 (Dense)            (None, 512)               524800    
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 dense_14 (Dense)            (None, 64)               

In [11]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
save = tf.keras.callbacks.ModelCheckpoint('best_model_ANN_0502.h5', monitor='val_loss', mode='min', save_best_only=True)

def scheduler(epoch):
    if epoch < 20:
        return 0.001
    else:
        return 0.001*math.exp(0.1*(10-epoch))

schedule = tf.keras.callbacks.LearningRateScheduler(scheduler)


callback = [es, save, schedule]

model3.fit(r_input,r_output, epochs=100, batch_size = 128, validation_split =0.1, callbacks=callback )    
model3.save("./weights_ANN.h5")

Epoch 1/10000
3516/3516 [==============================] - 26s 7ms/step - loss: 0.5197 - accuracy: 0.7764 - val_loss: 0.3732 - val_accuracy: 0.8369 - lr: 0.0010
Epoch 2/10000
3516/3516 [==============================] - 25s 7ms/step - loss: 0.3892 - accuracy: 0.8322 - val_loss: 0.3653 - val_accuracy: 0.8414 - lr: 0.0010
Epoch 3/10000
3516/3516 [==============================] - 24s 7ms/step - loss: 0.3731 - accuracy: 0.8395 - val_loss: 0.3910 - val_accuracy: 0.8338 - lr: 0.0010
Epoch 4/10000
3516/3516 [==============================] - 25s 7ms/step - loss: 0.3641 - accuracy: 0.8434 - val_loss: 0.3827 - val_accuracy: 0.8347 - lr: 0.0010
Epoch 5/10000
3516/3516 [==============================] - 25s 7ms/step - loss: 0.3590 - accuracy: 0.8452 - val_loss: 0.3438 - val_accuracy: 0.8515 - lr: 0.0010
Epoch 6/10000
3516/3516 [==============================] - 25s 7ms/step - loss: 0.3524 - accuracy: 0.8479 - val_loss: 0.3694 - val_accuracy: 0.8417 - lr: 0.0010
Epoch 7/10000
3516/3516 [=========

KeyboardInterrupt: 

In [13]:
test_input = np.load("./dataset3/testset/input1k_1.npy")
test_output = np.load("./dataset3/testset/output1k_1_num_class.npy")

#rpt_model=tf.keras.models.load_model("./best_model_effv2L_0413.h5")
rpt_model = tf.keras.models.load_model ('./best_model_ANN_0502.h5')
pred=rpt_model.predict(test_input)
true = 0
false = 0
pred_out = np.array([])
for i in range(pred.shape[0]):
    pred1=np.argmax(pred[i])
    pred_out=np.append(pred_out,pred1)
    if pred1==np.argmax(test_output[i]):
        true += 1
    else:
#        print(i)
        false +=1
acc = 100 * (true/(true+false))
print(true)
print(false)
print(acc,"%")

32/32 [==============================] - 0s 2ms/step
854
146
85.39999999999999 %
